In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import re

In [4]:
def define_url(checkin, checkout, adult, room, child):
    url = f"https://www.booking.com/searchresults.html?ss=Jordan&ssne=Jordan&ssne_untouched=Jordan&label=gen173nr-1FCAEoggI46AdIM1gEaHSIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGoAgO4Aorpla0GwAIB0gIkYWZkMWIxYTktNmE2Ny00MWU5LWExNWQtYzNlNDE5YzM3YjRh2AIF4AIB&sid=91e9df014d2c393cb40bdee65ae2a67a&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&latitude=31.97299&longitude=35.90846&checkin={checkin}&checkout={checkout}&group_adults={adult}&no_rooms={room}&group_children={child}&offset=0"
    return url

In [5]:
link = define_url('2024-03-01', '2024-03-30', 2, 1, 0)

link

'https://www.booking.com/searchresults.html?ss=Jordan&ssne=Jordan&ssne_untouched=Jordan&label=gen173nr-1FCAEoggI46AdIM1gEaHSIAQGYATG4ARfIAQzYAQHoAQH4AQKIAgGoAgO4Aorpla0GwAIB0gIkYWZkMWIxYTktNmE2Ny00MWU5LWExNWQtYzNlNDE5YzM3YjRh2AIF4AIB&sid=91e9df014d2c393cb40bdee65ae2a67a&aid=304142&lang=en-us&sb=1&src_elem=sb&src=searchresults&latitude=31.97299&longitude=35.90846&checkin=2024-03-01&checkout=2024-03-30&group_adults=2&no_rooms=1&group_children=0&offset=0'

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import time 

def get_element(parent, driver, xpath):
    for i in range(2):
        try: 
            WebDriverWait(driver, 5, ignored_exceptions=StaleElementReferenceException).until(
    EC.presence_of_element_located((By.XPATH, xpath))
    )
            el = parent.find_element(By.XPATH, xpath)
            driver.execute_script("arguments[0].scrollIntoView(true);", el)
            return el
        except:
            pass

names = []
locations = []
prices = []
ratings = []
reviews = []

def extract_hotel_info(hotel_element, driver):
    
    # Extracting hotel name
    name = get_element(hotel_element, driver, './/div[@data-testid="title"]').text
    names.append(name)
    
    # Extracting hotel location
    try:
        location = get_element(hotel_element, driver, './/span[@data-testid="address"]').text
        locations.append(location)
    except:
        location = 'Not found'
        locations.append(location)
    
    # Extracting hotel price
    price_xpath = './/span[@data-testid="price-and-discounted-price"]'
    price = get_element(hotel_element, driver, price_xpath).text.replace('JOD','')
    prices.append(price)
    
    #Extracting hotel rating
    try:
        rating = get_element(hotel_element, driver, './/div[@class="a3b8729ab1 d86cee9b25"]').text
        ratings.append(rating)
    except:
        rating = "No Rating"
        ratings.append(rating)
    
    #Extracting hotel reviews
    try:
        review = get_element(hotel_element, driver, './/div[@class="abf093bdfe f45d8e4c32 d935416c47"]').text.replace('reviews', '')
        reviews.append(review)
    except:
        review = "No Reviews"
        reviews.append(review)
    

def main():
    global link
    # Set the path to your WebDriver (e.g., chromedriver.exe)
    webdriver_path = 'D:/selenium/geckodriver.exe'

    # Initialize the Chrome browser
    driver = webdriver.Firefox(executable_path=webdriver_path)

    try:
        link = link.replace('offset=0', 'offset={}')

        hotels_data = []

        for i in range(0, 40):
            url = link.format(i*25)

            driver.get(url)
            time.sleep(5)
            # Replace 'hotel_elements_xpath' with the XPath that identifies each hotel in the list
            hotel_elements = driver.find_elements(By.XPATH, '//div[@aria-label="Property"]')
            try:
                el = driver.find_element(By.XPATH, "//div[@aria-label='Dismiss sign-in info.']")
                el.click()
            except:
                pass
            for hotel_element in hotel_elements:
                # Extract information from each hotel element
                try:
                    extract_hotel_info(hotel_element, driver)
                except AttributeError as e:
                    print(e)
                    print(hotel_element)
                    
            

    finally:
        # Close the browser window when done
        driver.quit()

if __name__ == "__main__":
    main()

C:\Users\User\AppData\Local\Temp\ipykernel_27384\364685651.py:68: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=webdriver_path)


In [25]:
hotels = pd.DataFrame({'Name': names, 'Location': locations,'Reviews': reviews, 'Rating': ratings, 'Price': prices})
hotels.head()

,Name,Location,Reviews,Rating,Price
0,Amman Marriott Hotel,Amman,813,8.4,"2,781"
1,Regency Palace Amman,Amman,"2,244",8.0,"2,610"
2,Corp Amman Hotel,Amman,"1,874",8.6,"1,549"
3,Grand Hotel,Amman,No Reviews,No Rating,"1,835"
4,الاردن عمان الشميساني,Amman,4,3.2,838
